In [1]:
import requests
import json
import re
import datetime
import numpy as np
import pandas as pd
from openpyxl import load_workbook
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 浏览器头
headers = {'content-type': 'application/json; charset=utf-8',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36'}

In [20]:
# codes = ['161725', '005827', '003095', '003096']
codes = ['161725']

# 昨天日期
yesterday = datetime.date.today() + datetime.timedelta(-1)

for code in codes:
    # url = "http://fundgz.1234567.com.cn/js/%s.js"%code
    url = "http://fund.eastmoney.com/pingzhongdata/%s.js"%code
    
    r = requests.get(url, headers=headers)
    # 返回信息
    content = r.text
    print(content[:2000])
    
    # 正则表达式
    pattern = r'Data_netWorthTrend = \[(.*?)\]'
    print(pattern)
    # 查找结果
    search = re.findall(pattern, content)
    print(search)
    data = json.loads(search[0])
    



/** * 测试数据 * @type {arry} *//*2022-07-21 17:17:24*/var ishb=false;/*基金或股票信息*/var fS_name = "招商中证白酒指数(LOF)A";var fS_code = "161725";/*原费率*/var fund_sourceRate="1.00";/*现费率*/var fund_Rate="0.10";/*最小申购金额*/var fund_minsg="10";/*基金持仓股票代码*/var stockCodes=["6008091","0008582","0005682","6005191","0023042","0005962","0007992","6033691","6007791","6035891"];/*基金持仓债券代码*/var zqCodes = "0196641,0196411,0196581";/*基金持仓股票代码(新市场号)*/var stockCodesNew =["1.600809","0.000858","0.000568","1.600519","0.002304","0.000596","0.000799","1.603369","1.600779","1.603589"];/*基金持仓债券代码（新市场号）*/var zqCodesNew = "1.019664,1.019641,1.019658";/*收益率*//*近一年收益率*/var syl_1n="-14.87";/*近6月收益率*/var syl_6y="-2.02";/*近三月收益率*/var syl_3y="11.67";/*近一月收益率*/var syl_1y="-0.74";/*股票仓位测算图*/var Data_fundSharesPositions = [[1655740800000,95.00],[1655827200000,95.00],[1655913600000,98.300],[1656000000000,97.3200],[1656259200000,95.00],[1656345600000,97.0900],[1656432000000,93.9200],[1656518400000,85.4100],[1656604800000,95.00],[16568640

JSONDecodeError: Extra data: line 1 column 60 (char 59)

In [8]:
codes = '161725'
url = "http://fundgz.1234567.com.cn/js/%s.js"%code

r = requests.get(url, headers=headers)
# 返回信息
content = r.text
print(content)
# 正则表达式
pattern = r'^jsonpgz\((.*)\)'
print(pattern)
# 查找结果
search = re.findall(pattern, content)
print(search)
data = json.loads(search[0])

jsonpgz({"fundcode":"161725","name":"招商中证白酒指数(LOF)A","jzrq":"2022-07-20","dwjz":"1.1968","gsz":"1.1838","gszzl":"-1.08","gztime":"2022-07-21 15:00"});
^jsonpgz\((.*)\)
['{"fundcode":"161725","name":"招商中证白酒指数(LOF)A","jzrq":"2022-07-20","dwjz":"1.1968","gsz":"1.1838","gszzl":"-1.08","gztime":"2022-07-21 15:00"}']


In [62]:
codes = ['161725', '005827', '003095', '003096']
# 获取表格对象
workbook = load_workbook('/Users/ayd/Desktop/基金数据买卖点.xlsx')
sheetnames = workbook.sheetnames
# 昨天日期
yesterday = datetime.date.today() + datetime.timedelta(-1)

for code in codes:
    url = "http://fundgz.1234567.com.cn/js/%s.js"%code
    # url = "http://fund.eastmoney.com/pingzhongdata/%s.js"%code
    
    r = requests.get(url, headers=headers)
    # 返回信息
    content = r.text
    # 正则表达式
    pattern = r'^jsonpgz\((.*)\)'
    # 查找结果
    search = re.findall(pattern, content)
    data = json.loads(search[0])
    
    # 创建sheet
    if data['name'] not in sheetnames:
        workbook.create_sheet(data['name'])
    sheet = workbook[data['name']]
    
    # 数据写入
    sheet['A1'] = '基金代码：' + data['fundcode']
    sheet['A2'] = '交易日期'
    sheet['B2'] = '单位净值'
    # 判断日期再写入
    all_date = [sheet['A'][a].value for a in range(len(sheet['A']))][2:]
    if str(yesterday) not in all_date:
        print('没有数据')
        sheet.insert_rows(idx=3, amount=1)
        sheet['A3'] = data['jzrq']
        sheet['B3'] = float(data['dwjz'])
        if len(sheet['A']) > 3:
            sheet['C3'] = float((sheet['B3'].value - sheet['B4'].value) / sheet['B4'].value * 100)

workbook.save('/Users/ayd/Desktop/基金数据买卖点.xlsx')